## Import Libraries

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

## MLFlow examples

In [ ]:
train_size = 0.8
penalty = "l2"
fit_intercept = False

In [ ]:
lr = LogisticRegression(
    penalty=penalty,
    fit_intercept=fit_intercept,
)

In [ ]:
X,y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=train_size)

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
pred = lr.predict(X_train)

In [ ]:
accuracy_train = (pred == y_train).mean()
accuracy_test = (lr.predict(X_test) == y_test).mean()

In [ ]:
thresholds = np.array([0.4, 0.3, 0.8])

## Logging To MLFlow

In [ ]:
import mlflow
from mlflow.models.signature import infer_signature

In [ ]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("ML_Experiment")

In [ ]:
with mlflow.start_run(run_name="SecondRun") as run:

    # === Logging Params ===
    mlflow.log_params({
        "train_size": train_size,
        "penalty": penalty,
        "fit_intercept": fit_intercept,
    })
    
    # === Logging Datasets ===
    mlflow.log_input(
        mlflow.data.from_numpy(X_train, name="X_train"),
        context="train"
    )
    mlflow.log_input(
        mlflow.data.from_numpy(y_train, name="y_train"),
        context="train"
    )
    mlflow.log_input(
        mlflow.data.from_numpy(pred, name="pred"),
        context="train_predict"
    )
    mlflow.log_input(
        mlflow.data.from_numpy(X_test, name="X_test"),
        context="test"
    )
    mlflow.log_input(
        mlflow.data.from_numpy(y_test, name="y_test"),
        context="test"
    )

    # === Logging Metrics ===
    mlflow.log_metrics({
        "accuracy_train":accuracy_train,
        "accuracy_test": accuracy_test,
    })

    # === Logging Model ===
    input_example = X_test[:1]
    signature = infer_signature(input_example, lr.predict(input_example))

    mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="Logistic Regression v2",
        signature=signature,
        input_example=input_example
    )